In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import librosa
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from pathlib import Path
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
!pwd

In [ ]:
!mkdir spectrograms

In [ ]:
df = pd.read_csv('/kaggle/input/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/patient_diagnosis.csv',
                      names=['Patient number', 'Diagnosis']
                  )

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(df['Diagnosis'])

In [ ]:
df_no_diagnosis = pd.read_csv('../input/respiratory-sound-database/demographic_info.txt', delimiter=' ',
                              names=['Patient number', 'Age', 'Sex' , 'Adult BMI (kg/m2)', 'Child Weight (kg)' , 'Child Height (cm)'],
                             )
df_no_diagnosis.head(5)

In [ ]:
df =  df_no_diagnosis.join(df.set_index('Patient number'), on='Patient number', how='left')
df.head(5)

In [ ]:
df.isna().sum()

In [ ]:
base_path = Path('../input/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/')

In [ ]:
df = df[["Patient number", "Sex", "Diagnosis"]]
df.isna().sum()
df.dropna(axis=0, inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.Diagnosis.value_counts()

In [ ]:
df = df.loc[df.Diagnosis != "Asthma"]
df = df.loc[df.Diagnosis != "LRTI"]
df = df.reset_index(drop=True)

In [ ]:
import soundfile as sf
samplerates = []
targets = []
filenames = []
for file in base_path.glob('audio_and_txt_files/*.wav'):
    _, sr = sf.read(file)
    filenames.append(str(file).split('/')[-1])
    targets.append(int(str(file).split('/')[-1][:3]))
    samplerates.append(sr)

In [ ]:
unique, counts = np.unique(samplerates, return_counts=True)
unique, counts

In [ ]:
patient_rate = []
for patient_id in df["Patient number"]:
    srs = True
    for file in base_path.glob(f'audio_and_txt_files/{patient_id}_*.wav'):
        _, sr = sf.read(file)
        if sr < 41000:
            srs = False
    patient_rate.append(srs)

In [ ]:
df = df[patient_rate]
df.reset_index(drop=True, inplace=True)

In [ ]:
import IPython
def display_wav(patient_id, target):
    patient_sounds = list(base_path.glob(f'audio_and_txt_files/{patient_id}*.wav'))
    test_wav = librosa.core.load(patient_sounds[0], sr=44100)[0]

    fig, ax = plt.subplots(5, 1, figsize=(20,18))
    ax[0].plot(test_wav)
    ax[0].set_title(f"Full {target} waveform")

    mel = librosa.feature.melspectrogram(test_wav, sr=44100, n_fft=512, hop_length=256, win_length=512, window='hann', n_mels=100)
    mel[mel < 1e-12] = 1e-12
    ax[1].imshow(np.log(mel)[:,:1000], origin='lower')
    ax[1].set_title(f"Slice of {target} mel spectrogram 512 fft")

    mel = librosa.feature.melspectrogram(test_wav, sr=44100, n_fft=1024, hop_length=256, win_length=1024, window='hann', n_mels=100)
    mel[mel < 1e-12] = 1e-12
    ax[2].imshow(np.log(mel)[:,:1000], origin='lower')
    ax[2].set_title(f"Slice of {target} mel spectrogram 1024 fft")

    mel = librosa.feature.melspectrogram(test_wav, sr=44100, n_fft=2048, hop_length=256, win_length=1024, window='hann', n_mels=100)
    mel[mel < 1e-12] = 1e-12
    ax[3].imshow(np.log(mel)[:,:1000], origin='lower')
    ax[3].set_title(f"Slice of {target} mel spectrogram 2048 fft")
    
    chroma = librosa.feature.chroma_stft(test_wav, sr=44100, n_fft=2048, hop_length=256, win_length=1024, window='hann')
    ax[4].imshow(chroma[:,:1000], origin='lower')
    ax[4].set_title(f"Slice of {target} chromagram 2048 fft")

In [ ]:
display_wav(102, "healthy")

In [ ]:
display_wav(104, "COPD")

In [ ]:
display_wav(103, "asthma")

In [ ]:
all_files = np.array(filenames)[np.array(samplerates) > 10000]

mics = []
sound_location = []
for i in range(len(all_files)):
    filename = all_files[i].split('/')[-1].split('_')
    mics.append(filename[4].replace(".wav", ""))
    sound_location.append(filename[2])

In [ ]:
all_mics = np.unique(np.array(mics))
all_locations = np.unique(np.array(sound_location))

In [ ]:
print(f"Unique microphones: {all_mics}")
print(f"Unique sound positions: {all_locations}")

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(all_mics)

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(all_locations)

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
file_lengths = []
for file in base_path.glob('audio_and_txt_files/*.wav'):
    data, sr = sf.read(file)
    if sr == 44100:
        file_lengths.append(len(data))

In [ ]:
sns.distplot(file_lengths, kde=False)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df, 
                                    random_state=42, test_size=0.26, 
                                    stratify=df["Diagnosis"])

In [ ]:
sns.countplot(X_train["Diagnosis"])
plt.title("Train target distribution")

In [ ]:
sns.countplot(X_test["Diagnosis"])
plt.title("Test target distribution")

In [ ]:
class Collater:
    def __init__(self, sample_seconds, repeat=1):
        self.sample_frames = int(sample_seconds * (44100 // 256))
        self.repeat = repeat
    
    def __call__(self, batch):
        new_batch = {
            "location": [],
            "gender": [],
            "mel": [],
            "chroma": [],
            "mic": [],
            "target": []
        }
        for entry in batch:
            new_batch["location"] += [entry["location"] for i in range(self.repeat)]
            new_batch["gender"]   += [entry["gender"] for i in range(self.repeat)]
            new_batch["mel"]      += [entry["mel"] for i in range(self.repeat)]
            new_batch["chroma"]   += [entry["chroma"] for i in range(self.repeat)]
            new_batch["mic"]      += [entry["mic"] for i in range(self.repeat)]
            new_batch["target"]   += [entry["target"] for i in range(self.repeat)]
        lengths = [entry.shape[1] for entry in new_batch["mel"]]
        
        start_frames = torch.tensor([np.random.randint(0, length - self.sample_frames - 5) for length in lengths])
        
        for i in range(len(new_batch["mel"])):
            new_batch["mel"][i] = new_batch["mel"][i][:, start_frames[i]:start_frames[i]+self.sample_frames]
            new_batch["chroma"][i] = new_batch["chroma"][i][:, start_frames[i]:start_frames[i]+self.sample_frames]
        
        new_batch["mel"] = torch.tensor(new_batch["mel"], dtype=torch.float32)
        new_batch["chroma"] = torch.tensor(new_batch["chroma"], dtype=torch.float32)
        new_batch["location"] = torch.tensor(new_batch["location"], dtype=torch.long)
        new_batch["gender"] = torch.tensor(new_batch["gender"], dtype=torch.long)
        new_batch["mic"] = torch.tensor(new_batch["mic"], dtype=torch.long)
        new_batch["target"] = torch.tensor(new_batch["target"], dtype=torch.long)
        
        return new_batch
            


class SoundDataset(Dataset):
    def __init__(self, df, mics, locations, genders, targets, sound_path, sample_seconds):
        self.all_mics = mics
        self.all_locations = locations
        self.all_genders = genders
        self.all_targets = targets
        self.sample_length = sample_seconds * 44100
        
        data = df.reset_index(drop=True)
        self.mics = []
        self.locations = []
        self.genders = []
        self.targets = []
        self.sound_len = []
        self.sound_files = []
        for i in range(len(df)):
            entry = df.iloc[i]
            patient_id = int(entry["Patient number"])
            for file in sound_path.glob(f"{patient_id}_*.wav"):
                self.sound_files.append(file)
                data, sr = sf.read(file)
                if sr != 44100:
                    print("Skipping ", file.stem)
                    continue
                self.sound_len.append(len(data))
                meta = str(file).split('/')[-1].split('_')
                self.mics.append(self.all_mics.index(meta[4].replace(".wav", "")))
                self.locations.append(self.all_locations.index(meta[2]))
                self.genders.append(self.all_genders.index(entry["Sex"]))
                self.targets.append(self.all_targets.index(entry["Diagnosis"]))
        
    def __getitem__(self, idx):
        sample_length = int(self.sample_length * 2.5)
        start_ind = np.random.randint(0, int(self.sound_len[idx] - sample_length - 5))
        waveform = sf.read(self.sound_files[idx], start=start_ind, frames=sample_length)[0]
        mel = librosa.feature.melspectrogram(waveform, sr=44100, n_fft=2048, hop_length=256, win_length=1024, window='hann', n_mels=100)
        mel[mel < 1e-12] = 1e-12

        chroma = librosa.feature.chroma_stft(waveform, sr=44100, n_fft=2048, hop_length=256, win_length=1024, window='hann')
        
        return {"mel": mel, 
                "chroma": chroma, 
                "gender": self.genders[idx], 
                "location": self.locations[idx], 
                "target": self.targets[idx], 
                "mic": self.mics[idx]}
    
    def __len__(self):
        return len(self.sound_files)

In [ ]:
import math
class PositionalEncoding(nn.Module):

    def __init__(self, hidden_dim, dropout_prob=0.1, max_len=5000):
        super().__init__()
        self.dropout_prob = dropout_prob

        pe = torch.zeros(max_len, hidden_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, hidden_dim, 2).float() * (-math.log(10000.0) / hidden_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return F.dropout(x, p=self.dropout_prob)

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, padding=(2,0), stride=1):
        super().__init__()
        self.triple_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels // 2, kernel_size=5, stride=stride, padding=padding),
            nn.BatchNorm2d(out_channels // 2),
            nn.ReLU(inplace=True),
            nn.Conv2d(
                out_channels // 2, out_channels // 2, kernel_size=5, stride=stride, padding=padding
            ),
            nn.BatchNorm2d(out_channels // 2),
            nn.ReLU(inplace=True),
            nn.Conv2d(
                out_channels // 2, out_channels, kernel_size=5, stride=stride, padding=padding
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.triple_conv(x)
    
class Model(nn.Module):
    def __init__(self, mel_channels, output_mel_channels, 
                 chroma_channels, output_chroma_channels, 
                 num_mics, mic_dim, num_sound_srcs, sound_src_dim,
                 num_genders, gender_dim, encoder_dim, 
                 output_dim, num_targets,
                 num_heads=8, num_layers=3, dropout=0.2):
        super().__init__()
        self.spectro_encoder = ConvBlock(mel_channels, output_mel_channels, padding=(0,0), stride=(2,1))
        
        self.chroma_encoder = ConvBlock(chroma_channels, output_chroma_channels, padding=(1,0))
        
        self.microphone_embedding = nn.Embedding(num_mics + 1, mic_dim)
        self.sound_source_embedding = nn.Embedding(num_sound_srcs + 1, sound_src_dim)
        
        self.gender_embedding = nn.Embedding(num_genders + 1, gender_dim)
        
        hidden_dim = 1948
        
        self.linear_projection = nn.Linear(hidden_dim, encoder_dim)
        
        self.pos_encoder = PositionalEncoding(encoder_dim, dropout)
        encoder_layer = nn.TransformerEncoderLayer(d_model=encoder_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.encoder_output_layer = nn.Linear(encoder_dim, output_dim)
        self.output_layer = nn.Linear(output_dim*2, num_targets)
        
    def forward(self, batch):
        
        spectro = self.spectro_encoder(batch["mel"].unsqueeze(1))
        
        chroma = self.chroma_encoder(batch["chroma"].unsqueeze(1))
        n_time = spectro.size(3)
        n_batch = spectro.size(0)
        
        mic_embed = self.microphone_embedding(batch["mic"]).unsqueeze(-1).expand(-1, -1, n_time).transpose(1,2)
        pos_embed = self.sound_source_embedding(batch["location"]).unsqueeze(-1).expand(-1, -1, n_time).transpose(1,2)
        gender_embed = self.gender_embedding(batch["gender"]).unsqueeze(-1).expand(-1, -1, n_time).transpose(1,2)
        
        chroma = chroma.transpose(1,3).reshape(n_batch, n_time, -1)
        spectro = spectro.transpose(1,3).reshape(n_batch, n_time, -1)
        
        x = torch.cat((chroma, spectro, mic_embed, pos_embed, gender_embed), 2)
        x = self.linear_projection(x)
        
        x = F.leaky_relu(x, negative_slope=0.2)
        
        x = self.pos_encoder(x)
        x = x.transpose(0, 1)
        x = self.transformer_encoder(x)
        x = x.transpose(0, 1)

        x = self.encoder_output_layer(x)
        x = F.leaky_relu(x, negative_slope=0.2)
        x = torch.cat([torch.mean(x, dim=1), torch.std(x, dim=1)], dim=1)

        x = self.output_layer(x)

        return x

In [ ]:
def to_cuda(batch):
    for key in batch.keys():
        batch[key] = batch[key].cuda()
    return batch

In [ ]:
@torch.no_grad()
def validate(model, val_data, criterion, batch_size):
    valset = SoundDataset(val_data, all_mics.tolist(), all_locations.tolist(), ["M", "F"], 
               ["COPD", "Healthy", "URTI", "Bronchiectasis", "Bronchiolitis", "Pneumonia"], base_path / "audio_and_txt_files", 6.0)
    collater = Collater(5.0, repeat=2)
    val_loader = DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=5, collate_fn=collater)
    
    mean_ce = 0.0
    model.eval()
    for i, batch in enumerate(val_loader):
        batch = to_cuda(batch)
        output = model(batch)
        loss = criterion(output, batch["target"])
        mean_ce += loss.item()
        
    mean_ce /= (i+1)
    model.train()
    return mean_ce

In [ ]:
batch_size = 16
criterion = torch.nn.CrossEntropyLoss()
lr = 3e-5
weight_decay=1e-8
epochs = 20
verbose_iter=10

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
model = Model(1, 128, 1, 128, len(all_mics), 8, len(all_locations), 16, 2, 4, 512, 128, 6, num_layers=3).cuda()
train_set = SoundDataset(X_train, all_mics.tolist(), all_locations.tolist(), ["M", "F"], 
               ["COPD", "Healthy", "URTI", "Bronchiectasis", "Bronchiolitis", "Pneumonia"], base_path / "audio_and_txt_files", 6.0)
collater = Collater(5.0, repeat=2)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=False, num_workers=5, collate_fn=collater)

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
iteration = 0
train_loss = []
val_loss = []
for epoch in range(1,epochs):
    for batch in train_loader:
        batch = to_cuda(batch)
        optimizer.zero_grad()
        
        output = model(batch)
        loss = criterion(output, batch["target"])
        
        loss.backward()
        optimizer.step()
        
        if iteration % verbose_iter == 0:
            print(f"{iteration}: Cross-entropy {loss.item()}")
        train_loss.append(loss.item())
        iteration += 1
    print("VALIDATING")
    val_ce = validate(model, X_test, criterion, batch_size)
    val_loss.append(val_ce)
    
    print(f"DONE EPOCH {epoch}: Train {train_loss[-1]} ;  Val {val_loss[-1]}")

In [ ]:
fig, ax = plt.subplots(figsize=(14,9))
plt.plot(train_loss)
plt.title("Train loss")

In [ ]:
fig, ax = plt.subplots(figsize=(14,9))
plt.plot(val_loss)
plt.title("Validation loss")

In [ ]:
valset = SoundDataset(X_test, all_mics.tolist(), all_locations.tolist(), ["M", "F"], 
           ["COPD", "Healthy", "URTI", "Bronchiectasis", "Bronchiolitis", "Pneumonia"], base_path / "audio_and_txt_files", 6.0)
collater = Collater(5.0, repeat=2)
val_loader = DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=5, collate_fn=collater)

targets = []
predictions = []

model.eval()
with torch.no_grad():
    for i, batch in enumerate(val_loader):
        batch = to_cuda(batch)
        output = model(batch)
        
        targets.append(batch["target"].cpu().numpy())
        predictions.append(output.cpu().numpy())

In [ ]:
new_targets = []
for entry in targets:
    new_targets += entry.tolist()

In [ ]:
new_preds = []
for pred in predictions:
    new_preds += np.argmax(pred, axis=1).tolist()

In [ ]:
target_classes = ["COPD", "Healthy", "URTI", "Bronchiectasis", "Bronchiolitis", "Pneumonia"]

In [ ]:
from sklearn.metrics import confusion_matrix
fig, ax = plt.subplots(figsize=(13,10))
sns.heatmap(confusion_matrix(new_targets, new_preds), annot=True, annot_kws={"size": 16}, cmap='ocean_r')
ax.set_xticklabels(target_classes)
ax.set_yticklabels(target_classes)

In [ ]:
from sklearn.preprocessing import label_binarize
y_test = label_binarize(new_targets, classes=[0, 1, 2, 3, 4, 5])

In [ ]:
predicted_proba = []
for proba in predictions:
    predicted_proba += nn.Softmax()(torch.tensor(proba)).cpu().numpy().tolist()
predicted_proba = np.array(predicted_proba)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(target_classes)):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], predicted_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
fig, ax = plt.subplots(figsize=(14,9))
for i in range(len(target_classes)):
    plt.plot(fpr[i], tpr[i], label=f'{i} ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")